# Exploring Algorithm Transformations in `cubing_algs`

**A comprehensive guide for developers and speedcubing hobbyists**

The `cubing_algs` library provides a powerful transformation system that allows you to modify, optimize, and convert algorithms between different notations and representations. This notebook explores the modular transformation functions available in `cubing_algs.transform`, demonstrating their practical applications in algorithm development and analysis.

## What You'll Learn

- **Core transformation concepts** - Understanding how transformations work and can be chained
- **Move optimization** - Compression techniques to reduce move count and improve efficiency  
- **Notation conversion** - Converting between standard notation, SiGN notation, wide moves, and slice moves
- **Algorithmic manipulation** - Mirror inversions, symmetries, rotations, and grip handling
- **Performance optimization** - Real-world applications for algorithm development and analysis
- **Chain composition** - Combining multiple transformations for complex algorithm modifications

## Prerequisites

- Basic understanding of Rubik's cube notation
- Familiarity with the `Algorithm` and `Move` classes (see previous notebooks)
- Python programming fundamentals

Let's dive into the world of algorithm transformations!

In [1]:
# Import the core components
from cubing_algs import Algorithm
from cubing_algs.transform import *
import time

# Set up some example algorithms for demonstration
sexy_move = Algorithm.parse_moves("R U R' U'")
t_perm = Algorithm.parse_moves("R U R' F' R U R' U' R' F R2 U' R'")
j_perm = Algorithm.parse_moves("R U R' F' R U R' U' R' F R2 U' R' U'")
sune = Algorithm.parse_moves("R U R' U R U2 R'")
y_perm = Algorithm.parse_moves("F R U' R' U' R U R' F' R U R' U' R' F R F'")

print("=== Example Algorithms ===")
print(f"Sexy Move: {sexy_move}")
print(f"T-Perm: {t_perm}")
print(f"J-Perm: {j_perm}")
print(f"Sune: {sune}")
print(f"Y-Perm: {y_perm}")
print(f"\nOriginal T-Perm metrics: {t_perm.metrics}")

=== Example Algorithms ===
Sexy Move: R U R' U'
T-Perm: R U R' F' R U R' U' R' F R2 U' R'
J-Perm: R U R' F' R U R' U' R' F R2 U' R' U'
Sune: R U R' U R U2 R'
Y-Perm: F R U' R' U' R U R' F' R U R' U' R' F R F'

Original T-Perm metrics: {'pauses': 0, 'rotations': 0, 'outer_moves': 13, 'inner_moves': 0, 'htm': 13, 'qtm': 14, 'stm': 13, 'etm': 13, 'rtm': 0, 'qstm': 14, 'generators': ['R', 'U', 'F']}


## 1. Core Transformation Concepts

The transformation system in `cubing_algs` follows a functional approach where each transformation function takes an `Algorithm` and returns a new `Algorithm`. This design enables:

1. **Immutability** - Original algorithms are never modified
2. **Composability** - Transformations can be chained together
3. **Modularity** - Each transformation has a single responsibility

### The Transform Method

All transformations can be applied using the `Algorithm.transform()` method, which supports chaining multiple transformations:

In [2]:
# Basic transformation examples
from cubing_algs.transform.mirror import mirror_moves
from cubing_algs.transform.size import compress_moves

# Single transformation
mirrored_sexy = sexy_move.transform(mirror_moves)
print(f"Original: {sexy_move}")
print(f"Mirrored: {mirrored_sexy}")

# Chained transformations
optimized_tperm = t_perm.transform(compress_moves, mirror_moves, compress_moves)
print(f"\nOriginal T-Perm: {t_perm}")
print(f"Compressed → Mirrored → Compressed: {optimized_tperm}")

# Direct function calls (equivalent to transform)
direct_mirror = mirror_moves(sexy_move)
print(f"\nDirect function call: {direct_mirror}")
print(f"Transform method: {mirrored_sexy}")
print(f"Results identical: {direct_mirror == mirrored_sexy}")

Original: R U R' U'
Mirrored: U R U' R'

Original T-Perm: R U R' F' R U R' U' R' F R2 U' R'
Compressed → Mirrored → Compressed: R U R2 F' R U R U' R' F R U' R'

Direct function call: U R U' R'
Transform method: U R U' R'
Results identical: True


### Fixpoint Transformations

An important feature of the transform system is the ability to apply transformations repeatedly until no further changes occur - this is called reaching a "fixpoint". The `transform()` method supports this with the `to_fixpoint=True` parameter:

```python
algorithm.transform(compress_moves, to_fixpoint=True)
```

This applies the transformation iteratively until the algorithm no longer changes, ensuring complete optimization.

In [3]:
from cubing_algs.transform.optimize import *
# Fixpoint transformation examples
print("=== Fixpoint Transformations ===")

# Create an algorithm that requires multiple compression passes
complex_alg = Algorithm.parse_moves("R R' U U' R R R U U U F F' F' F R R R R")
print(f"Complex algorithm: {complex_alg}")

# Single pass compression
single_pass = complex_alg.transform(compress_moves)
print(f"Single compression pass: {single_pass}")

# Fixpoint compression - keeps applying until no more changes
fixpoint_result = complex_alg.transform(compress_moves, to_fixpoint=True)
print(f"Fixpoint compression: {fixpoint_result}")

# Demonstrate why fixpoint is useful with chained optimizations
messy_alg = Algorithm.parse_moves("R U R' R U R' U R R R' U' U R' U R U' R' R' R U R'")
print(f"\nMessy algorithm: {messy_alg}")

# Without fixpoint - single application
without_fixpoint = messy_alg.transform(
    optimize_do_undo_moves,
    optimize_double_moves, 
    optimize_triple_moves,
    optimize_repeat_three_moves
)
print(f"Without fixpoint: {without_fixpoint}")

# With fixpoint - applied until stable
with_fixpoint = messy_alg.transform(
    optimize_do_undo_moves,
    optimize_double_moves,
    optimize_triple_moves, 
    optimize_repeat_three_moves,
    to_fixpoint=True
)
print(f"With fixpoint: {with_fixpoint}")

# Show iterative optimization manually to understand fixpoint behavior
print(f"\nManual iterative optimization:")
current = messy_alg
iteration = 0
while True:
    iteration += 1
    previous = current
    current = current.transform(
        optimize_do_undo_moves,
        optimize_double_moves,
        optimize_triple_moves,
        optimize_repeat_three_moves
    )
    print(f"Iteration {iteration}: {current} ({len(current)} moves)")
    
    if current == previous:  # Fixpoint reached
        print(f"Fixpoint reached after {iteration} iterations")
        break
    
    if iteration > 10:  # Safety check
        print("Max iterations reached")
        break

# Practical example: Complex algorithm cleanup
scrambled_pattern = Algorithm.parse_moves("R U R' U R U R' U' R' U R R R' U' U R' R R U R'")
print(f"\nScrambled pattern: {scrambled_pattern}")

# Compare normal vs fixpoint optimization
normal_opt = scrambled_pattern.transform(compress_moves)
fixpoint_opt = scrambled_pattern.transform(compress_moves, to_fixpoint=True)

print(f"Normal optimization: {normal_opt} ({len(normal_opt)} moves)")
print(f"Fixpoint optimization: {fixpoint_opt} ({len(fixpoint_opt)} moves)")
print(f"Additional reduction: {len(normal_opt) - len(fixpoint_opt)} moves")

=== Fixpoint Transformations ===
Complex algorithm: R R' U U' R R R U U U F F' F' F R R R R
Single compression pass: R' U'
Fixpoint compression: R' U'

Messy algorithm: R U R' R U R' U R R R' U' U R' U R U' R' R' R U R'
Without fixpoint: R U2 R' U2 R U' R' U R'
With fixpoint: R U2 R' U2 R U' R' U R'

Manual iterative optimization:
Iteration 1: R U2 R' U2 R U' R' U R' (9 moves)
Iteration 2: R U2 R' U2 R U' R' U R' (9 moves)
Fixpoint reached after 2 iterations

Scrambled pattern: R U R' U R U R' U' R' U R R R' U' U R' R R U R'
Normal optimization: R U R' U R U R' U' R' U R2 U R' (13 moves)
Fixpoint optimization: R U R' U R U R' U' R' U R2 U R' (13 moves)
Additional reduction: 0 moves


## 2. Move Optimization and Compression

One of the most practical applications of transformations is algorithm optimization. The `size` module provides powerful compression techniques that can significantly reduce move count by eliminating redundancies.

### Available Compression Functions

- `compress_moves()` - Apply all optimization techniques iteratively
- `expand_moves()` - Convert double moves to two single moves
- Individual optimizers: `optimize_double_moves`, `optimize_triple_moves`, `optimize_do_undo_moves`, `optimize_repeat_three_moves`

In [4]:
# Compression demonstration with various redundant algorithms
from cubing_algs.transform.size import compress_moves, expand_moves

# Create some algorithms with obvious redundancies
redundant_alg = Algorithm.parse_moves("R R R U U F F' U' U' R'")
double_alg = Algorithm.parse_moves("R2 U2 F2")
complex_redundant = Algorithm.parse_moves("R U R' U R U R' U' R' U R R R'")

print("=== Move Compression Examples ===")
print(f"Redundant algorithm: {redundant_alg}")
print(f"After compression: {redundant_alg.transform(compress_moves)}")
print(f"Moves reduced: {len(redundant_alg)} → {len(redundant_alg.transform(compress_moves))}")

print(f"\nDouble moves: {double_alg}")
print(f"Expanded: {double_alg.transform(expand_moves)}")

print(f"\nComplex redundant: {complex_redundant}")
print(f"After compression: {complex_redundant.transform(compress_moves)}")

# Step-by-step optimization
step1 = optimize_repeat_three_moves(complex_redundant)
step2 = optimize_double_moves(step1) 
step3 = optimize_do_undo_moves(step2)

print(f"\nStep-by-step optimization:")
print(f"Original: {complex_redundant}")
print(f"After triple optimization: {step1}")
print(f"After double optimization: {step2}")
print(f"After do-undo optimization: {step3}")

# Real-world example: Optimizing scrambled algorithms
scrambled_sune = Algorithm.parse_moves("R U R' R R' U R U2 R' R'")
print(f"\nScrambled Sune: {scrambled_sune}")
print(f"Optimized: {scrambled_sune.transform(compress_moves)}")
print(f"Should be: {sune}")

=== Move Compression Examples ===
Redundant algorithm: R R R U U F F' U' U' R'
After compression: R2
Moves reduced: 10 → 1

Double moves: R2 U2 F2
Expanded: R R U U F F

Complex redundant: R U R' U R U R' U' R' U R R R'
After compression: R U R' U R U R' U' R' U R

Step-by-step optimization:
Original: R U R' U R U R' U' R' U R R R'
After triple optimization: R U R' U R U R' U' R' U R R R'
After double optimization: R U R' U R U R' U' R' U R2 R'
After do-undo optimization: R U R' U R U R' U' R' U R2 R'

Scrambled Sune: R U R' R R' U R U2 R' R'
Optimized: R U R' U R U2 R2
Should be: R U R' U R U2 R'


## 3. Notation Conversion and Style

The library supports conversion between different move notations and styles, enabling compatibility with various cubing communities and tools.

### Notation Systems

- **Standard Notation**: Traditional FRULDB notation (R, U, F, etc.)
- **SiGN Notation**: Alternative notation system for advanced algorithms
- **Wide Moves**: Rw, Uw style moves for larger cube layers
- **Slice Moves**: M, E, S moves for middle layers

In [5]:
# Notation conversion examples
from cubing_algs.transform.sign import sign_moves, unsign_moves
from cubing_algs.transform.wide import unwide_slice_moves, unwide_rotation_moves, rewide_moves
from cubing_algs.transform.slice import unslice_wide_moves, unslice_rotation_moves, reslice_moves

print("=== Notation Conversion Examples ===")

# Standard to SiGN notation
print(f"Standard T-Perm: {t_perm}")
sign_tperm = t_perm.transform(sign_moves)
print(f"SiGN notation: {sign_tperm}")
back_to_standard = sign_tperm.transform(unsign_moves)
print(f"Back to standard: {back_to_standard}")
print(f"Round-trip successful: {t_perm == back_to_standard}")

# Wide move conversions
wide_alg = Algorithm.parse_moves("Rw U Rw' Fw R F'")
print(f"\nWide move algorithm: {wide_alg}")
print(f"Expanded to slice: {wide_alg.transform(unwide_slice_moves)}")
print(f"Expanded to rotation: {wide_alg.transform(unwide_rotation_moves)}")

# Slice move conversions  
slice_alg = Algorithm.parse_moves("M' U' M U2 M' U' M")
print(f"\nSlice algorithm: {slice_alg}")
print(f"Converted to wide: {slice_alg.transform(unslice_wide_moves)}")
print(f"Converted to rotation: {slice_alg.transform(unslice_rotation_moves)}")

# Complex conversion chain
complex_chain = wide_alg.transform(
    unwide_slice_moves,    # Wide → slice + outer
    reslice_moves,         # Recombine to slice where possible
    compress_moves         # Optimize the result
)
print(f"\nComplex conversion chain:")
print(f"Original: {wide_alg}")
print(f"Final: {complex_chain}")

=== Notation Conversion Examples ===
Standard T-Perm: R U R' F' R U R' U' R' F R2 U' R'
SiGN notation: R U R' F' R U R' U' R' F R2 U' R'
Back to standard: R U R' F' R U R' U' R' F R2 U' R'
Round-trip successful: True

Wide move algorithm: Rw U Rw' Fw R F'
Expanded to slice: R M' U R' M F S R F'
Expanded to rotation: L x U L' x' B z R F'

Slice algorithm: M' U' M U2 M' U' M
Converted to wide: r R' U' r' R U2 r R' U' r' R
Converted to rotation: L R' x U' L' R x' U2 L R' x U' L' R x'

Complex conversion chain:
Original: Rw U Rw' Fw R F'
Final: R M' U R' M F S R F'


## 4. Algorithmic Manipulation and Symmetries

Beyond optimization and notation conversion, transformations enable sophisticated algorithmic manipulations including mirroring, symmetries, and mathematical operations.

### Mirror Operations

The `mirror_moves()` function creates the inverse of an algorithm by reversing the move order and inverting each move:

In [6]:
# Mirror and symmetry operations
from cubing_algs.transform.symmetry import *

print("=== Mirror and Symmetry Operations ===")

# Mirror operations (inverse algorithms)
print(f"Original Sune: {sune}")
mirrored_sune = sune.transform(mirror_moves)
print(f"Mirrored Sune: {mirrored_sune}")

# Verify that algorithm + mirror = identity (should return to solved)
combined = sune + mirrored_sune
print(f"Sune + Mirror: {combined}")
print(f"Compressed: {combined.transform(compress_moves)}")

# Symmetry transformations
print(f"\nSymmetry transformations on T-Perm:")
print(f"Original: {t_perm}")
print(f"M-symmetry: {t_perm.transform(symmetry_m_moves)}")
print(f"S-symmetry: {t_perm.transform(symmetry_s_moves)}")
print(f"E-symmetry: {t_perm.transform(symmetry_e_moves)}")
print(f"C-symmetry: {t_perm.transform(symmetry_c_moves)}")

# Practical application: Creating algorithm variants
oll_case = Algorithm.parse_moves("R U R' U R U2 R'")  # Sune
print(f"\nCreating OLL variants:")
print(f"Base case: {oll_case}")
variants = [
    oll_case.transform(symmetry_m_moves),
    oll_case.transform(symmetry_s_moves), 
    oll_case.transform(symmetry_e_moves),
    oll_case.transform(mirror_moves)
]
for i, variant in enumerate(variants, 1):
    print(f"Variant {i}: {variant}")

=== Mirror and Symmetry Operations ===
Original Sune: R U R' U R U2 R'
Mirrored Sune: R U2 R' U' R U' R'
Sune + Mirror: R U R' U R U2 R' R U2 R' U' R U' R'
Compressed: 

Symmetry transformations on T-Perm:
Original: R U R' F' R U R' U' R' F R2 U' R'
M-symmetry: L' U' L F L' U' L U L F' L2 U L
S-symmetry: R' U' R B R' U' R U R B' R2 U R
E-symmetry: R' D' R F R' D' R D R F' R2 D R
C-symmetry: L U L' B' L U L' U' L' B L2 U' L'

Creating OLL variants:
Base case: R U R' U R U2 R'
Variant 1: L' U' L U' L' U2 L
Variant 2: R' U' R U' R' U2 R
Variant 3: R' D' R D' R' D2 R
Variant 4: R U2 R' U' R U' R'


## 5. Rotation Optimization and Grip Handling

Cube rotations are often used in algorithms but can slow down execution. The rotation and degrip modules provide tools for optimizing rotation sequences and removing unnecessary grip changes.

### Rotation Optimization

The `rotation` module optimizes cube rotations by:

In [7]:
# Rotation and grip optimization
from cubing_algs.transform.rotation import *
from cubing_algs.transform.degrip import *

print("=== Rotation and Grip Optimization ===")

# Algorithm with redundant rotations
rotation_heavy = Algorithm.parse_moves("x R U R' x' y R U R' y' x2 y2 z2")
print(f"Rotation-heavy algorithm: {rotation_heavy}")

# Remove final rotations (commonly used in speedsolving)
cleaned = rotation_heavy.transform(remove_final_rotations)
print(f"Final rotations removed: {cleaned}")

# Compress rotation sequences
compressed_rotations = rotation_heavy.transform(compress_rotations)
print(f"Compressed rotations: {compressed_rotations}")

# Split moves and rotations for analysis
moves, rotations = split_moves_final_rotations(rotation_heavy)
print(f"Moves part: {moves}")
print(f"Rotations part: {rotations}")

# Degrip operations (remove grip changes)
grip_algorithm = Algorithm.parse_moves("R U R' y x R U R' x'")
print(f"\nGrip algorithm: {grip_algorithm}")
print(f"X-axis degripped: {grip_algorithm.transform(degrip_x_moves)}")
print(f"Fully degripped: {grip_algorithm.transform(degrip_full_moves)}")

# Practical speedsolving example
cfop_alg = Algorithm.parse_moves("R U R' F' R U R' U' R' F R2 U' R' y R U R' U'")
print(f"\nCFOP algorithm: {cfop_alg}")
optimized_cfop = cfop_alg.transform(
    compress_moves,
    degrip_full_moves,
    compress_moves,
    remove_final_rotations,
)
print(f"Optimized for speed: {optimized_cfop}")
print(f"Move reduction: {len(cfop_alg)} → {len(optimized_cfop)}")

=== Rotation and Grip Optimization ===
Rotation-heavy algorithm: x R U R' x' y R U R' y' x2 y2 z2
Final rotations removed: x R U R' x' y R U R'
Compressed rotations: x R U R' x' y R U R' y'
Moves part: x R U R' x' y R U R'
Rotations part: y' x2 y2 z2

Grip algorithm: R U R' y x R U R' x'
X-axis degripped: R U R' y R F R' x' x
Fully degripped: R U R' B R B' z x z'

CFOP algorithm: R U R' F' R U R' U' R' F R2 U' R' y R U R' U'
Optimized for speed: R U R' F' R U R' U' R' F R2 U' R' B U B' U'
Move reduction: 18 → 17


## 6. Timing, Pauses, and AUF Handling

For algorithm timing analysis and competition preparation, several transforms handle temporal aspects and algorithmic conveniences.

### Timing and Pause Management

In [8]:
# Timing and pause handling
from cubing_algs.transform.timing import untime_moves
from cubing_algs.transform.pause import unpause_moves, pause_moves
from cubing_algs.transform.auf import remove_auf_moves

print("=== Timing and Pause Management ===")

# Create a timed algorithm
timed_alg = Algorithm.parse_moves("R@100 U@200 R'@300 U'@400")
print(f"Timed algorithm: {timed_alg}")
print(f"Remove timing: {timed_alg.transform(untime_moves)}")

# Pause insertion based on timing gaps
slow_alg = Algorithm.parse_moves("R@0 U@50 R'@1000 U'@1100")  # Big gap before R'
paused = slow_alg.transform(pause_moves(speed=100, factor=3))
print(f"\nSlow algorithm: {slow_alg}")
print(f"With pauses: {paused}")

# Remove pauses
print(f"Remove pauses: {paused.transform(unpause_moves)}")

# AUF (Adjust Upper Face) handling
auf_algorithm = Algorithm.parse_moves("U R U R' U R U2 R' U'")
print(f"\nAUF algorithm: {auf_algorithm}")
no_auf = auf_algorithm.transform(remove_auf_moves)
print(f"AUF removed: {no_auf}")

# Complex timing scenario
complex_timing = Algorithm.parse_moves("R@0 U@100 R'@200 .@500 U'@600 R@1500 U@1600")
print(f"\nComplex timing: {complex_timing}")
processed = complex_timing.transform(
    pause_moves(speed=200, factor=2),
    unpause_moves,  # Remove existing pauses
    untime_moves    # Clean up timing
)
print(f"Processed: {processed}")

=== Timing and Pause Management ===
Timed algorithm: R@100 U@200 R'@300 U'@400
Remove timing: R U R' U'

Slow algorithm: R@0 U@50 R'@1000 U'@1100
With pauses: R@0 U@50 .@525 R'@1000 U'@1100
Remove pauses: R@0 U@50 R'@1000 U'@1100

AUF algorithm: U R U R' U R U2 R' U'
AUF removed: R U R' U R U2 R'

Complex timing: R@0 U@100 R'@200 .@500 U'@600 R@1500 U@1600
Processed: R U R' U' R U


## 7. Advanced Transformation Techniques

Let's explore advanced patterns and techniques that demonstrate the full power of the transformation system.

### Custom Transformation Functions

You can create your own transformation functions following the same pattern:

In [9]:
# Custom transformation functions
from cubing_algs.move import Move

def custom_filter_faces(faces_to_keep):
    """Create a transform that keeps only specified faces."""
    def _filter_transform(algorithm):
        filtered_moves = []
        for move in algorithm:
            if move.is_pause or move.is_rotation_move or move.base_move in faces_to_keep:
                filtered_moves.append(move)
        return Algorithm(filtered_moves)
    return _filter_transform

def custom_substitute_moves(substitutions):
    """Create a transform that substitutes specific moves."""
    def _substitute_transform(algorithm):
        new_moves = []
        for move in algorithm:
            if str(move) in substitutions:
                new_moves.extend(Algorithm.parse_moves(substitutions[str(move)]))
            else:
                new_moves.append(move)
        return Algorithm(new_moves)
    return _substitute_transform

# Test custom transformations
print("=== Custom Transformations ===")

test_alg = Algorithm.parse_moves("R U R' F' R U2 R' U' F L' U L")
print(f"Original: {test_alg}")

# Filter to only R and U moves
ru_only = custom_filter_faces(['R', 'U'])
filtered = test_alg.transform(ru_only)
print(f"R/U moves only: {filtered}")

# Substitute R with R2
substitutions = {'R': 'R2', "R'": "R2'"}
substitute_r = custom_substitute_moves(substitutions)
substituted = test_alg.transform(substitute_r)
print(f"R substituted: {substituted}")

# Complex custom pipeline
complex_custom = test_alg.transform(
    substitute_r,           # Substitute R moves
    compress_moves,         # Optimize 
    ru_only,               # Filter to R/U
    compress_moves          # Final optimization
)
print(f"Complex custom: {complex_custom}")

# Performance comparison
def transform_with_timing(algorithm, *transforms):
    start = time.perf_counter()
    result = algorithm.transform(*transforms)
    end = time.perf_counter()
    return result, (end - start) * 1000  # milliseconds

# Time various transformation chains
long_alg = Algorithm.parse_moves(" ".join([str(t_perm)] * 10))  # Repeat T-perm 10 times
print(f"\nPerformance test on {len(long_alg)}-move algorithm:")

transformations = [
    ("compress_moves", compress_moves),
    ("mirror + compress", mirror_moves, compress_moves),
    ("full optimization", compress_moves, remove_final_rotations, degrip_full_moves, compress_moves),
    ("custom chain", substitute_r, ru_only, compress_moves)
]

for name, *transforms in transformations:
    result, timing = transform_with_timing(long_alg, *transforms)
    print(f"{name}: {len(long_alg)} → {len(result)} moves ({timing:.2f}ms)")

=== Custom Transformations ===
Original: R U R' F' R U2 R' U' F L' U L
R/U moves only: R U R' R U2 R' U' U
R substituted: R2 U R2 F' R2 U2 R2 U' F L' U L
Complex custom: R2 U' R2

Performance test on 130-move algorithm:
compress_moves: 130 → 85 moves (2.06ms)
mirror + compress: 130 → 85 moves (2.08ms)
full optimization: 130 → 85 moves (1.67ms)
custom chain: 130 → 0 moves (1.53ms)


## 8. Real-World Applications and Best Practices

Let's explore practical applications of transformations in speedcubing and algorithm development.

### Algorithm Development Workflow

In [10]:
# Real-world application examples
print("=== Real-World Applications ===")

# 1. Algorithm comparison and analysis
def analyze_algorithm(alg, name="Algorithm"):
    """Comprehensive algorithm analysis using transformations."""
    print(f"\n{name} Analysis:")
    print(f"Original: {alg}")
    print(f"Length: {len(alg)} moves")
    print(f"Metrics: HTM={alg.metrics['htm']}, QTM={alg.metrics['qtm']}")
    
    # Try various optimizations
    compressed = alg.transform(compress_moves)
    degripped = alg.transform(degrip_full_moves, compress_moves)
    no_rotations = alg.transform(remove_final_rotations, compress_moves)
    
    print(f"Compressed: {compressed} ({len(compressed)} moves)")
    print(f"Degripped: {degripped} ({len(degripped)} moves)")
    print(f"No final rotations: {no_rotations} ({len(no_rotations)} moves)")
    
    # Find best optimization
    optimizations = [
        ("Original", alg),
        ("Compressed", compressed),
        ("Degripped", degripped),
        ("No rotations", no_rotations)
    ]
    
    best = min(optimizations, key=lambda x: len(x[1]))
    print(f"Best: {best[0]} with {len(best[1])} moves")
    return best[1]

# Analyze different algorithm types
pll_t = Algorithm.parse_moves("R U R' F' R U R' U' R' F R2 U' R'")
oll_sune = Algorithm.parse_moves("R U R' U R U2 R'")
f2l_case = Algorithm.parse_moves("R U' R' F R F'")

best_pll = analyze_algorithm(pll_t, "T-Perm")
best_oll = analyze_algorithm(oll_sune, "Sune")
best_f2l = analyze_algorithm(f2l_case, "F2L Case")

# 2. Algorithm set generation
print("\n=== Algorithm Set Generation ===")
base_alg = Algorithm.parse_moves("R U R' U'")
print(f"Base algorithm: {base_alg}")

# Generate all symmetry variants
symmetries = [
    ("Original", lambda x: x),
    ("Mirror", mirror_moves),
    ("M-symmetry", symmetry_m_moves),
    ("S-symmetry", symmetry_s_moves),
    ("E-symmetry", symmetry_e_moves)
]

print("Generating algorithm set:")
alg_set = {}
for name, transform in symmetries:
    variant = base_alg.transform(transform, compress_moves)
    alg_set[name] = variant
    print(f"{name}: {variant}")

# Check for duplicates
unique_algs = set(str(alg) for alg in alg_set.values())
print(f"Unique algorithms: {len(unique_algs)} out of {len(alg_set)}")

# 3. Competition preparation
print("\n=== Competition Preparation ===")
comp_alg = Algorithm.parse_moves("R U R' F' R U R' U' R' F R2 U' R' y R U R' U'")
print(f"Competition algorithm: {comp_alg}")

# Optimize for speed
speed_optimized = comp_alg.transform(
    degrip_full_moves,       # Minimize grip changes
    compress_moves,          # Optimize move sequence
    remove_final_rotations,  # Remove unnecessary rotations
    untime_moves,            # Remove timing for clean display
    unpause_moves,           # Remove pauses
)

print(f"Speed optimized: {speed_optimized}")
print(f"Improvement: {len(comp_alg)} → {len(speed_optimized)} moves")
print(f"HTM reduction: {comp_alg.metrics['htm']} → {speed_optimized.metrics['htm']}")

# Ergonomic analysis
def ergonomic_score(alg):
    """Simple ergonomic scoring based on move types."""
    score = 0
    for move in alg:
        if move.base_move in ['R', 'U']:
            score += 1  # Easy moves
        elif move.base_move in ['L', 'D']:
            score -= 1  # Harder moves
        elif move.is_rotation_move:
            score -= 2  # Rotations slow you down
    return score

print(f"Ergonomic scores:")
print(f"Original: {ergonomic_score(comp_alg)}")
print(f"Optimized: {ergonomic_score(speed_optimized)}")

=== Real-World Applications ===

T-Perm Analysis:
Original: R U R' F' R U R' U' R' F R2 U' R'
Length: 13 moves
Metrics: HTM=13, QTM=14
Compressed: R U R' F' R U R' U' R' F R2 U' R' (13 moves)
Degripped: R U R' F' R U R' U' R' F R2 U' R' (13 moves)
No final rotations: R U R' F' R U R' U' R' F R2 U' R' (13 moves)
Best: Original with 13 moves

Sune Analysis:
Original: R U R' U R U2 R'
Length: 7 moves
Metrics: HTM=7, QTM=8
Compressed: R U R' U R U2 R' (7 moves)
Degripped: R U R' U R U2 R' (7 moves)
No final rotations: R U R' U R U2 R' (7 moves)
Best: Original with 7 moves

F2L Case Analysis:
Original: R U' R' F R F'
Length: 6 moves
Metrics: HTM=6, QTM=6
Compressed: R U' R' F R F' (6 moves)
Degripped: R U' R' F R F' (6 moves)
No final rotations: R U' R' F R F' (6 moves)
Best: Original with 6 moves

=== Algorithm Set Generation ===
Base algorithm: R U R' U'
Generating algorithm set:
Original: R U R' U'
Mirror: U R U' R'
M-symmetry: L' U' L U
S-symmetry: R' U' R U
E-symmetry: R' D' R D
Unique

## 9. Best Practices and Guidelines

Based on the comprehensive exploration above, here are key recommendations for using transformations effectively:

### Performance Guidelines

1. **Order matters**: Apply more specific transforms before general ones
2. **Chain wisely**: Group related transforms together for better performance
3. **Test thoroughly**: Always verify that transformations preserve algorithm correctness

### Common Transformation Patterns

```python
# Standard optimization pipeline
optimized = algorithm.transform(
    degrip_full_moves,      # Remove grip changes
    remove_final_rotations, # Clean up trailing rotations  
    compress_moves          # Final cleanup
)

# Notation conversion pipeline
converted = algorithm.transform(
    unwide_slice_moves,     # Expand wide moves
    reslice_moves,          # Recombine to slices
    compress_moves          # Optimize result
)

# Analysis pipeline for algorithm comparison
variants = [
    algorithm,                             # Original
    algorithm.transform(mirror_moves),     # Mirror
    algorithm.transform(symmetry_m_moves), # M-symmetry
    algorithm.transform(compress_moves),   # Optimized
]
```

### When to Use Each Transform

- **compress_moves**: Always use for optimization and cleanup
- **mirror_moves**: For creating inverse algorithms and analysis
- **symmetry_***: For generating algorithm variants and sets
- **unwide_/reslice_**: For notation compatibility and analysis
- **degrip_***: For speedsolving optimization
- **remove_final_rotations**: For competition algorithms
- **sign_moves/unsign_moves**: For notation conversion between communities

## 10. Summary and Next Steps

The `cubing_algs` transformation system provides a powerful, modular approach to algorithm manipulation that serves both developers and speedcubing enthusiasts. Key takeaways:

### What We Covered

1. **Core Concepts**: Functional, immutable transformation system with method chaining
2. **Optimization**: Comprehensive move compression and algorithmic improvements
3. **Notation Systems**: Seamless conversion between standard, SiGN, wide, and slice notations
4. **Algorithmic Manipulation**: Mirroring, symmetries, and mathematical operations
5. **Rotation Handling**: Optimization of cube rotations and grip management
6. **Timing Analysis**: Tools for competition preparation and performance analysis
7. **Custom Extensions**: Framework for building your own transformations
8. **Real-World Applications**: Practical workflows for algorithm development

### Key Benefits

- **Modularity**: Each transform has a single responsibility
- **Composability**: Transformations can be chained naturally
- **Performance**: Optimized implementations with caching where appropriate
- **Flexibility**: Support for custom transformations and workflows
- **Reliability**: Immutable operations preserve original algorithms

### Next Steps

- Explore the VCube visualization system for seeing transformation effects
- Build custom transformation pipelines for your specific use cases
- Integrate transformations into algorithm analysis and development workflows
- Experiment with the metrics system to quantify transformation benefits

The transformation system forms the backbone of algorithm manipulation in `cubing_algs`, enabling everything from simple optimizations to complex algorithmic research. Whether you're optimizing algorithms for competition, analyzing algorithm sets, or developing new cubing tools, these transformations provide the building blocks for sophisticated algorithm manipulation.